In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import nengo

In [45]:
model = nengo.Network()
with model:
    
    external = nengo.Node(0.5)
    internal = nengo.Ensemble(n_neurons=100, dimensions=1)
    
    state = nengo.Ensemble(n_neurons=300, dimensions=2)
    
    #state2 = nengo.Ensemble(n_neurons=300, dimensions=2)
    
    y = nengo.Ensemble(n_neurons=100, dimensions=1)
    
    nengo.Connection(external, state[0])
    nengo.Connection(internal, state[1])
    
    nengo.Connection(state, y, function=lambda x: (x[0]-x[1])**2)
    
    q = nengo.networks.EnsembleArray(n_neurons=100, n_ensembles=2)
    error = nengo.networks.EnsembleArray(n_neurons=100, n_ensembles=2)
    c_up = nengo.Connection(state, q.input[0], function=lambda x: 0, learning_rule_type=nengo.PES(learning_rate=1e-3))
    c_dn = nengo.Connection(state, q.input[1], function=lambda x: 0, learning_rule_type=nengo.PES(learning_rate=1e-3))
    inh = -1
    
    choice = nengo.networks.EnsembleArray(n_neurons=100, n_ensembles=2, encoders=nengo.dists.Choice([[1]]),
                                     intercepts=nengo.dists.Uniform(0,1))
    nengo.Connection(q.output, choice.input)
    nengo.Connection(choice.output, choice.input, transform=[[0,inh],[inh,0]])
    nengo.Connection(choice.output[0], error.all_ensembles[1].neurons, transform=-3*np.ones((100, 1)))
    nengo.Connection(choice.output[1], error.all_ensembles[0].neurons, transform=-3*np.ones((100, 1)))
    nengo.Connection(q.output, error.input)
    
    dydt = nengo.Node(None, size_in=1)
    nengo.Connection(y, dydt, synapse=0.005, transform=-10)
    nengo.Connection(y, dydt, synapse=0.050, transform=10)
    nengo.Connection(dydt, error.input, transform=-np.ones((2,1)))
    nengo.Connection(error.output[0], c_up.learning_rule)
    nengo.Connection(error.output[1], c_dn.learning_rule)
    
    nengo.Connection(internal, internal, synapse=0.1)
    nengo.Connection(choice.output, internal, synapse=0.1, transform=[[-0.2,0.2]])
    
    bias = nengo.Node(0.5)
    nengo.Connection(bias, choice.input, transform=np.ones((2,1)))
    
    
    
    
    


In [46]:
import nengo_gui.jupyter
nengo_gui.jupyter.InlineGUI(model, cfg='minimize.cfg')

c:\users\terry\documents\github\nengo_gui\nengo_gui\jupyter.py:70: ConfigReuseWarning: Reusing config. Only the most recent visualization will update the config.
  "Reusing config. Only the most recent visualization will "
